# 处理搜狐数据，去掉article content中带有html标签

In [21]:
import sys
sys.path.append('../utils')
import sql_util as sql
from bs4 import BeautifulSoup
import re

In [ ]:
# 查询所有数据
result_data = sql.queryall('select * from sj_sohu_no_html')
print('数据总数: {}', len(result_data))
# 循环处理数据，去掉每一条数据中的article_content html标签
# 正则表达式过滤标签<>...</> or <.../>
count = 0
for element in result_data:
    if count % 5000 == 0:
        print('process %d data...' % count)
    temp = re.compile(r'<[^>]*>', re.S)
    temp = temp.sub('', element.get('article_content'))
    temp = str.replace(temp, '\n', '').strip()
    finally_result = sql.execute("update sj_sohu_no_html set article_content = %s where id = %s", (temp, element.get('id')))
    if finally_result == 0:
        print('faild...')
    count += 1
print('process finally......')

process 0 data...
process 5000 data...
process 10000 data...
process 15000 data...


In [19]:
# 查询所有数据
result_data = sql.queryall('select * from sj_sohu_no_html limit 1')
# print(result_data[0].get('article_content').strip())
print(str.replace(result_data[0].get('article_content'), '\n', '').strip())

参考消息网1月24日报道海外媒体称，1月23日晚一名男游客玩航拍期间，疑似无人机突然故障无信号，撞击澳门大三巴牌坊后“坠落”二层窗口，当地保安报警处理。 据《澳门日报》1月24日报道，23日晚8点半许，澳门警方接到报案信息，称大三巴牌坊发生航拍无人机毁损事件，警员到场向保安及涉事的23岁内地男游客了解情况。据称有人玩航拍期间，疑似无人机突然无信号并失去动力，“降落”到牌坊二层中央窗口，现场人指称航拍机曾与牌坊发生碰撞后坠落。事发后现场被封锁，消防人员亦到场戒备，随后将无人机取下。 澳门警方表示，涉事航拍无人机未向当地申请航拍准照，涉案游客被带走调查。 另据香港《明报》网站1月24日报道，澳门文化局24日再次派人员详细检查大三巴牌坊，包括现场检视、照片比对及使用3D扫描作数据分析等，经过严谨检查后，确认大三巴牌坊没有受损。 报道称，澳门方面未来会加强公众对文物建筑安全的保护意识，并呼吁市民及游客使用航拍机时，除需遵守相关规定外，更应确保文物建筑安全，若令建筑受损，需负刑事责任。                                声明：该文观点仅代表作者本人，搜狐号系信息发布平台，搜狐仅提供信息存储空间服务。


In [26]:
print ("PI = %d" % 3)

PI = 3


In [ ]:
result_data = sql.queryall('select * from sj_sohu_no_html')
print('数据总数: {}', len(result_data))